# 1 - Imports

In [324]:
%reload_ext autoreload
%autoreload 2

In [325]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parents[0]))

In [326]:
import pandas as pd
import numpy as np

In [327]:
from src.utils import config, io, countries

# 1 - Build Catalogs

In [328]:
from src.extraction import dbnomics_client

In [329]:
client = dbnomics_client.DBnomicsClient()

## 1.1 Provider Catalog

### 1.1.1 - Build Provider Datasets Catalog (Discontinued, Now Hand Picked List)

In [270]:
from src.metadata import build_provider_dataset_catalog

In [282]:
providers = [
    'IMF', 'WB', 'OECD', 'BIS', 'UNCTAD', 'FAO', 'ILO', 'FH'
]

In [283]:
datasets_df = build_provider_dataset_catalog.get_all_datasets_df(client, providers)
datasets_df

,index,Provider Id,Provider Name,Provider Region,Dataset Id,Dataset Name,Subgroup 1 Id,Subgroup 1 Name,Subgroup 2 Id,Subgroup 2 Name
0,0,IMF,International Monetary Fund,World,AFRREO,Sub-Saharan Africa Regional Economic Outlook (...,NaN,NaN,NaN,NaN
1,1,IMF,International Monetary Fund,World,APDREO,Asia and Pacific Regional Economic Outlook (AP...,NaN,NaN,NaN,NaN
2,2,IMF,International Monetary Fund,World,BOP,Balance of Payments (BOP),NaN,NaN,NaN,NaN
3,3,IMF,International Monetary Fund,World,BOPAGG,"Balance of Payments (BOP), World and Regional ...",NaN,NaN,NaN,NaN
4,4,IMF,International Monetary Fund,World,CDIS,Coordinated Direct Investment Survey (CDIS),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2106,2562,ILO,International Labour Organization,World,WBL_TWBL_SEX_EDU_WBL_NB,"Work-based learners by sex, education and type...",WBL,Work-based learning,NaN,NaN
2107,2563,ILO,International Labour Organization,World,WBL_XVET_SEX_EDU_NB,Working-age population with vocational educati...,WBL,Work-based learning,NaN,NaN
2108,2564,ILO,International Labour Organization,World,WBL_3WBL_SEX_WBL_RT,Youth participation rate in work-based learnin...,WBL,Work-based learning,NaN,NaN
2109,2565,FH,Freedom House,World,FIW,"All data, Freedom in the world since 2013",NaN,NaN,NaN,NaN


In [284]:
datasets_df['Provider Id'].value_counts()

Provider Id
ILO       1071
OECD       758
IMF        106
FAO         85
UNCTAD      62
WB          14
BIS         13
FH           2
Name: count, dtype: int64

In [285]:
build_provider_dataset_catalog.TOTAL_AVOIDED

4550

In [ ]:
# io.save_csv(datasets_df, config.METADATA_DIR / 'provider_datasets.csv')

### 1.1.2 Prune Provider Datasets

In [441]:
datasets_df = io.load_csv(config.METADATA_DIR / 'provider_datasets_selected.csv')
datasets_df

,index,Provider Id,Provider Name,Provider Region,Dataset Id,Dataset Name,Subgroup 1 Id,Subgroup 1 Name,Subgroup 2 Id,Subgroup 2 Name
0,2,IMF,International Monetary Fund,World,BOP,Balance of Payments (BOP),NaN,NaN,NaN,NaN
1,4,IMF,International Monetary Fund,World,CDIS,Coordinated Direct Investment Survey (CDIS),NaN,NaN,NaN,NaN
2,5,IMF,International Monetary Fund,World,COFER,Currency Composition of Official Foreign Excha...,NaN,NaN,NaN,NaN
3,6,IMF,International Monetary Fund,World,CPI,Consumer Price Index (CPI),NaN,NaN,NaN,NaN
4,8,IMF,International Monetary Fund,World,DOT,Direction of Trade Statistics (DOTS),NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
96,1646,ILO,International Labour Organization,World,SDG_F881_SEX_MIG_RT,SDG indicator 8.8.1 - Fatal occupational injur...,INJ,Occupational injuries,NaN,NaN
97,1647,ILO,International Labour Organization,World,SDG_N881_SEX_MIG_RT,SDG indicator 8.8.1 - Non-fatal occupational i...,INJ,Occupational injuries,NaN,NaN
98,1648,ILO,International Labour Organization,World,INJ_FATL_ECO_NB,Cases of fatal occupational injury by economic...,INJ,Occupational injuries,NaN,NaN
99,2563,ILO,International Labour Organization,World,WBL_XVET_SEX_EDU_NB,Working-age population with vocational educati...,WBL,Work-based learning,NaN,NaN


In [442]:
def check_valid_frequency(dataset_dict):
    if 'FREQ' in dataset_dict['dataset']['dimensions_values_labels'].keys():
        freq_dict = dataset_dict['dataset']['dimensions_values_labels']['FREQ']

        if 'A' in freq_dict.keys():
            return True
        else:
            print('No annual frequency data:', freq_dict)
            return False
    else:
        print('FREQ not in keys', dataset_dict['dataset']['dimensions_values_labels'].keys())
        return False

In [443]:
datasets_df['Provider Id'].value_counts()

Provider Id
IMF       21
UNCTAD    20
ILO       20
FAO       16
WB        10
BIS       10
OECD       3
FH         1
Name: count, dtype: int64

In [444]:
providers

['IMF', 'ILO', 'UNCTAD', 'FAO', 'WB', 'BIS', 'OECD', 'FH']

In [501]:
from src.metadata import build_indicator_catalog

indicators_catalog = build_indicator_catalog.get_indicator_catalog(datasets_df, client)
indicators_catalog

Provider: UNCTAD
	Dataset: MPCADIOEAIA
Percent of OECD Countries covered: 0.51 %
Not Enough countries covered for: UNCTAD MPCADIOEAIA
	Dataset: UCPIA21
Couldn't find Country dimension values
Not found Indicator Dimension in keys dict_keys(['commodity', 'frequency', 'measure'])
:::New Number of Indicators: 0
	Dataset: FDIIAOFASA
Percent of OECD Countries covered: 0.51 %
Not Enough countries covered for: UNCTAD FDIIAOFASA
	Dataset: TAUPA


KeyboardInterrupt: 

In [491]:
d = client.list_series('ILO', 'MST_TUNE_SEX_AGE_CCT_NB')
d

{'_meta': {'args': {'align_periods': False,
   'dataset_code': 'MST_TUNE_SEX_AGE_CCT_NB',
   'dimensions': {},
   'facets': False,
   'format': 'json',
   'limit': 1000,
   'metadata': True,
   'observations': False,
   'offset': 0,
   'provider_code': 'ILO',
   'q': '',
   'series_code': None},
  'version': '22.1.17'},
 'dataset': {'attributes_labels': {'OBSV_STATUS': 'Observation status'},
  'attributes_values_labels': {'OBSV_STATUS': {'A': 'Adjusted',
    'B': 'Break in series',
    'C': 'Confidential',
    'E': 'Estimate',
    'I': 'Imputation',
    'M': 'Model-based extrapolation',
    'P': 'Provisional',
    'R': 'Real value',
    'S': 'Not significant',
    'U': 'Unreliable'}},
  'code': 'MST_TUNE_SEX_AGE_CCT_NB',
  'dimensions_codes_order': ['ref_area',
   'source',
   'classif1',
   'classif2',
   'sex',
   'frequency'],
  'dimensions_labels': {'classif1': 'Classification 1',
   'classif2': 'Classification 2',
   'frequency': 'Frequency',
   'ref_area': 'Reference area',
   's

In [492]:
d.keys()

dict_keys(['_meta', 'dataset', 'errors', 'provider', 'series'])

In [493]:
d['dataset'].keys()

dict_keys(['attributes_labels', 'attributes_values_labels', 'code', 'dimensions_codes_order', 'dimensions_labels', 'dimensions_values_labels', 'dir_hash', 'indexed_at', 'name', 'nb_series', 'provider_code', 'provider_name'])

In [494]:
d['dataset']['dimensions_values_labels'].keys()

dict_keys(['classif1', 'classif2', 'frequency', 'ref_area', 'sex', 'source'])

In [498]:
d['dataset']['dimensions_values_labels']['sex']

{'SEX_F': 'Sex: Female', 'SEX_M': 'Sex: Male', 'SEX_T': 'Sex: Total'}

In [499]:
d['series'].keys()

dict_keys(['docs', 'limit', 'num_found', 'offset'])

In [500]:
d['series']['docs'][0]

{'dataset_code': 'MST_TUNE_SEX_AGE_CCT_NB',
 'dataset_name': 'Unemployment by sex, age and citizenship (thousands)',
 'dimensions': {'classif1': 'AGE_AGGREGATE_TOTAL',
  'classif2': 'CCT_CIT_CITIZEN',
  'frequency': 'A',
  'ref_area': 'AGO',
  'sex': 'SEX_F',
  'source': 'AA_835'},
 'indexed_at': '2024-02-12T04:24:30.950Z',
 'provider_code': 'ILO',
 'series_code': 'AGO.AA_835.AGE_AGGREGATE_TOTAL.CCT_CIT_CITIZEN.SEX_F.A',
 'series_name': 'Angola – PC - Population Census (AA_835) – Age (Aggregate bands): Total – Citizenship: National citizens – Sex: Female – Annual'}

In [372]:
dataset_dict = client.list_series('IMF', 'WEO:2008-04')
dataset_dict

{'_meta': {'args': {'align_periods': False,
   'dataset_code': 'WEO:2008-04',
   'dimensions': {},
   'facets': False,
   'format': 'json',
   'limit': 1000,
   'metadata': True,
   'observations': False,
   'offset': 0,
   'provider_code': 'IMF',
   'q': '',
   'series_code': None},
  'version': '22.1.17'},
 'dataset': {'code': 'WEO:2008-04',
  'dimensions_codes_order': ['weo-country', 'weo-subject', 'unit'],
  'dimensions_labels': {'unit': 'Unit',
   'weo-country': 'WEO Country',
   'weo-subject': 'WEO Subject'},
  'dimensions_values_labels': {'unit': {'annual_pcent_change': 'Annual percent change',
    'current_international_dollar': 'Current international dollar',
    'idx': 'Index',
    'idx_2000_100': 'Index, 2000=100',
    'national_currency': 'National currency',
    'national_currency_per_current_international_dollar': 'National currency per current international dollar',
    'pcent': 'Percent',
    'pcent_gdp': 'Percent of GDP',
    'pcent_potential_gdp': 'Percent of potentia

In [ ]:
dataset_dict

{'_meta': {'args': {'align_periods': False,
   'dataset_code': 'COFER',
   'dimensions': {},
   'facets': False,
   'format': 'json',
   'limit': 1000,
   'metadata': True,
   'observations': False,
   'offset': 0,
   'provider_code': 'IMF',
   'q': '',
   'series_code': None},
  'version': '22.1.17'},
 'dataset': {'attributes_labels': {'OBS_STATUS': 'Observation Status (incl. Confidentiality)',
   'TIME_FORMAT': 'Time format',
   'UNIT_MULT': 'Scale'},
  'attributes_values_labels': {'TIME_FORMAT': {'P1D': 'Daily',
    'P1M': 'Monthly',
    'P1Y': 'Annual',
    'P3M': 'Quarterly',
    'P6M': 'Bi-annual',
    'P7D': 'Weekly'},
   'UNIT_MULT': {'0': 'Units',
    '1': 'Tens',
    '10': 'Ten Billions',
    '11': 'Hundred Billions',
    '12': 'Trillions',
    '13': 'Ten Trillions',
    '14': 'Hundred Trillions',
    '15': 'Quadrillions',
    '2': 'Hundreds',
    '3': 'Thousands',
    '4': 'Ten Thousands',
    '5': 'Hundred Thousands',
    '6': 'Millions',
    '7': 'Ten Millions',
    '8': '

In [309]:
dataset_dict['dataset']['dimensions_values_labels']['FREQ']

{'A': 'Annual',
 'B': 'Bi-annual',
 'D': 'Daily',
 'M': 'Monthly',
 'Q': 'Quarterly',
 'W': 'Weekly'}

## 1.3 Build Indicators Catalog

In [ ]:
import sdmx

In [ ]:
IMF_DATA = sdmx.Client('IMF_DATA')

In [ ]:
IMF_DATA.data('CPI')

/Users/hippolytegrandet/Desktop/Dev/country_risk_rating/.venv/lib/python3.9/site-packages/sdmx/rest/common.py:367: UserWarning: 'agency_id' argument is redundant for data queries
  getattr(self, f"handle_{query_type}")()
xml.Reader got no structure=… argument for StructureSpecificData


KeyboardInterrupt: 

In [ ]:
IMF_DATA.data

ValueError: can't determine a reader for response content type None

In [ ]:
data_msg = IMF_DATA.data('CPI', key='USA+CAN.CPI.CP01.IX.M', params={'startPeriod': 2018})
cpi_df = sdmx.to_pandas(data_msg)
cpi_df

xml.Reader got no structure=… argument for StructureSpecificData


TIME_PERIOD  INDEX_TYPE  COICOP_1999  TYPE_OF_TRANSFORMATION  COUNTRY  FREQUENCY  COMMON_REFERENCE_PERIOD  OVERLAP  SCALE  ACCESS_SHARING_LEVEL  SECURITY_CLASSIFICATION
2018-M01     CPI         CP01         IX                      CAN      M          2002A                    OL       0      PUBLIC_OPEN           PUB                        142.600000
2018-M02     CPI         CP01         IX                      CAN      M          2002A                    OL       0      PUBLIC_OPEN           PUB                        142.300000
2018-M03     CPI         CP01         IX                      CAN      M          2002A                    OL       0      PUBLIC_OPEN           PUB                        141.300000
2018-M04     CPI         CP01         IX                      CAN      M          2002A                    OL       0      PUBLIC_OPEN           PUB                        141.900000
2018-M05     CPI         CP01         IX                      CAN      M          2002A            

In [ ]:
from extraction import dbnomics_client
client = dbnomics_client.DBnomicsClient()

In [ ]:
BIS_datasets = client.list_datasets("BIS")

In [ ]:
provider_info = {}
provider_info[datasets['provider']['code']] = BIS_datasets['provider']['region'] + ' Region. ' + BIS_datasets['provider']['name']
provider_info

{'BIS': 'World Region. Bank for International Settlements'}

In [ ]:
BIS_datasets.keys()

dict_keys(['_meta', 'category_tree', 'provider'])

In [ ]:
datasets_BIS = {}

def get_datasets_pairs(l):
    datsets_descriptions = {}

    subname = ''
    for e in l:
        print(e['name'])
        subname = e['name']
        subsubname = ''
        for f in e['children']:
            subsubname = f['name']
            subsubsubname = ''
            for g in f['children']:
                subsubsubname = g['name']
                code = g['code']

                datsets_descriptions[code] = subname + subsubname + subsubsubname

    return datsets_descriptions

datasets_BIS_mapping = get_datasets_pairs(BIS_datasets['category_tree'])
datasets_BIS_mapping

BISWEB_CATSCHEME
        
Dataportal Category Scheme
        


{'WS_LBS_D_PUB': 'Dataportal Category Scheme\n        International banking\n          BIS locational banking',
 'WS_CBS_PUB': 'Dataportal Category Scheme\n        International banking\n          BIS consolidated banking',
 'WS_DEBT_SEC2_PUB': 'Dataportal Category Scheme\n        Debt securities\n          BIS international debt securities (BIS-compiled)',
 'WS_OTC_DERIV2': 'Dataportal Category Scheme\n        Derivatives\n          OTC derivatives outstanding',
 'WS_DER_OTC_TOV': 'Dataportal Category Scheme\n        Derivatives\n          OTC derivatives turnover',
 'WS_XTD_DERIV': 'Dataportal Category Scheme\n        Derivatives\n          Exchange traded derivatives',
 'WS_GLI': 'Dataportal Category Scheme\n        Global liquidity\n          Global liquidity indicators',
 'WS_TC': 'Dataportal Category Scheme\n        Credit\n          BIS long series on total credit',
 'WS_CREDIT_GAP': 'Dataportal Category Scheme\n        Credit\n          BIS credit-to-GDP gaps',
 'WS_DSR': 'Data

In [ ]:
len(datasets_BIS_mapping)

41

In [ ]:
client.list_series('BIS', 'WS_CBTA')

{'_meta': {'args': {'align_periods': False,
   'dataset_code': 'WS_CBTA',
   'dimensions': {},
   'facets': False,
   'format': 'json',
   'limit': 1000,
   'metadata': True,
   'observations': False,
   'offset': 0,
   'provider_code': 'BIS',
   'q': '',
   'series_code': None},
  'version': '22.1.17'},
 'dataset': {'attributes_labels': {'BREAKS': 'Breaks',
   'COLLECTION': 'Collection Indicator',
   'COLLECTION_DETAIL': 'Collection explanation detail',
   'COMMENT_DSET': '',
   'COMMENT_TS': 'Series comment',
   'COMPILING_ORG': 'Compiling agency',
   'CONF_STATUS': 'Confidentiality - status',
   'DATA_COMP': 'Data compilation',
   'DECIMALS': 'Decimals',
   'DISS_ORG': 'Data dissemination agency',
   'FISCAL_YEAR': 'Fiscal year',
   'METHOD_REF': 'Methodology reference',
   'OBS_PRE_BREAK': 'Observation pre-break value',
   'OBS_STATUS': 'Observation status',
   'SUPP_INFO_BREAKS': 'Supplemental information and breaks',
   'TIME_FORMAT': 'Time format',
   'UNIT_MULT': 'Unit multipli

In [ ]:
print('category_tree')
print(len(datasets['category_tree']))
for e in datasets['category_tree']:
        print('\t', e['name'])
        print('\t', e['code'], len(e['children']))
        for f in e['children']:
            print('\t\t', f['name'])
            print('\t\t', f['code'], len(f['children']))

            for g in f['children']:
                print('\t\t\t', g['name'])
                print('\t\t\t', g['code'])
                

category_tree
2
	 BISWEB_CATSCHEME
        
	 BISWEB_CATSCHEME 10
		 International banking
          
		 WEBSTATS_BANKING 2
			 BIS locational banking
			 WS_LBS_D_PUB
			 BIS consolidated banking
			 WS_CBS_PUB
		 Debt securities
          
		 WEBSTATS_SEC 1
			 BIS international debt securities (BIS-compiled)
			 WS_DEBT_SEC2_PUB
		 Derivatives
          
		 WEBSTATS_DER 3
			 OTC derivatives outstanding
			 WS_OTC_DERIV2
			 OTC derivatives turnover
			 WS_DER_OTC_TOV
			 Exchange traded derivatives
			 WS_XTD_DERIV
		 Global liquidity
          
		 WEBSTATS_GLI 1
			 Global liquidity indicators
			 WS_GLI
		 Total credit
          
		 WEBSTATS_TOTCRED 1
			 BIS long series on total credit
			 WS_TC
		 Credit gaps
          
		 WEBSTATS_CREDGAP 1
			 BIS credit-to-GDP gaps
			 WS_CREDIT_GAP
		 Debt service ratio
          
		 WEBSTATS_DSR 1
			 BIS debt service ratio
			 WS_DSR
		 Property prices
          
		 WEBSTATS_PP 1
			 Selected residential property prices
			 WS_SPP
		 Cons

In [ ]:
client.list_series('BIS', 'CBTA')

HTTPError: 404 Client Error: NOT FOUND for url: https://api.db.nomics.world/v22/series/BIS/CBTA?limit=1000&offset=0

In [ ]:
client.list_series('BIS', 'WS_CBTA')

{'_meta': {'args': {'align_periods': False,
   'dataset_code': 'WS_CBTA',
   'dimensions': {},
   'facets': False,
   'format': 'json',
   'limit': 1000,
   'metadata': True,
   'observations': False,
   'offset': 0,
   'provider_code': 'BIS',
   'q': '',
   'series_code': None},
  'version': '22.1.17'},
 'dataset': {'attributes_labels': {'BREAKS': 'Breaks',
   'COLLECTION': 'Collection Indicator',
   'COLLECTION_DETAIL': 'Collection explanation detail',
   'COMMENT_DSET': '',
   'COMMENT_TS': 'Series comment',
   'COMPILING_ORG': 'Compiling agency',
   'CONF_STATUS': 'Confidentiality - status',
   'DATA_COMP': 'Data compilation',
   'DECIMALS': 'Decimals',
   'DISS_ORG': 'Data dissemination agency',
   'FISCAL_YEAR': 'Fiscal year',
   'METHOD_REF': 'Methodology reference',
   'OBS_PRE_BREAK': 'Observation pre-break value',
   'OBS_STATUS': 'Observation status',
   'SUPP_INFO_BREAKS': 'Supplemental information and breaks',
   'TIME_FORMAT': 'Time format',
   'UNIT_MULT': 'Unit multipli

In [ ]:
def list_all_series(client, provider, dataset):
    all_series = []
    offset = 0
    limit = 1000

    while True:
        data = client.list_series(provider, dataset, limit, offset)
        series = data.get("series", [])
        if not series:
            break
        all_series.extend(series)
        offset += limit

    return all_series


In [ ]:
list_all_series(client, 'BIS', 'WS_CPMI_CT1')

HTTPError: 400 Client Error: BAD REQUEST for url: https://api.db.nomics.world/v22/series/BIS/WS_CPMI_CT1?limit=1000&offset=101000

# 2 - Querying LLM